In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import interpolate
import matplotlib.pyplot as plt
import astropy.constants as const
from astropy.io.votable import parse
from configparser import ConfigParser
import matplotlib.patheffects as PathEffects

In [2]:
#@|+++++++++++++++++
#@|Exoplanet catalog
#@|+++++++++++++++++

catalog = 'Exoplanet.eu'    #NEA, Exoplanet.eu, or PlanetS 
precision_radius = 30       #Maximum relative precision in radius (%). If 0, no threshold is applied
precision_mass = 0          #Maximum relative precision in mass (%).   If 0, no threshold is applied

In [3]:
#@|+++++++++++++++++++++++++++++
#@|+++Graphic configuration+++++
#@|+++++++++++++++++++++++++++++

legend_or_box = 'box'            #legend or box | Include the planet names in the legend or in a text box. 
text = 'desert, ridge, savanna'  #desert, ridge, and/or savanna | Overplot the text of the regimes in the figure
color_text = 'lightblue'         #Text color
boundaries = 'desert, ridge'     #desert, and/or ridge | Plot the boundaries of the desert, ridge, or both
color_boundaries = 'black'       #Color of the computed boundaries  
style_boundaries = 'dashed'      #Style of the boundaries
color_map = 'magma_r'            #Color map of the density contours
plot_name = 'example2'           #Name of the plot to be saved in the output folder

In [ ]:
#@|++++++++++++++++++++++++++++++++++
#@|++++++My planets (example 2)++++++
#@|++++++++++++++++++++++++++++++++++

#@|++++++HATS-17b+++++++
name_p1 = 'HATS-17 b'   #Name of planet 1
color_p1 = 'green'      #Color of planet 1
p_p1 = 16.254611        #days      (Brahm et al. 2016)
r_p1 = 8.71             #R_earth   (Brahm et al. 2016)
r_p1_err_up = 0.63      #R_earth    (Brahm et al. 2016) 
r_p1_err_down = 0.63    #R_earth   (Brahm et al. 2016)

dis_x_p1 = 0            #Location of a text box in terms of distance from the planet (X-axis)
dis_y_p1 = 1.3          #Location of a text box in terms of distance from the planet (Y-axis)

In [ ]:
if catalog == 'NEA':

    list_complete = sorted(os.listdir('catalog_data/NEA/'))
    df = pd.read_csv('catalog_data/NEA/'+list_complete[-1], comment = "#")

if catalog == 'Exoplanet.eu':
    
    file = os.listdir('catalog_data/Exoplanet.eu/')[0]
    df = pd.read_csv('catalog_data/Exoplanet.eu/'+file)
    
    df = df.rename(columns={'orbital_period':'pl_orbper','radius':'pl_rade',\
                            'radius_error_min':'pl_radeerr2','radius_error_max':'pl_radeerr1' })            
if catalog == 'PlanetS':
 
    votable_PlanetS = parse("catalog_data/PlanetS/PLANETS.vot")
    table_PlanetS = votable_PlanetS.get_first_table().to_table()
    df = table_PlanetS.to_pandas()
    
    df = df.rename(columns={'Orbital Period [days]':'pl_orbper',\
                        'Planet Radius [Rjup]':'pl_rade','Planet Radius - Upper Unc [Rjup]':'pl_radeerr1',\
                        'Planet Radius - Lower Unc [Rjup]':'pl_radeerr2' })
    

In [ ]:
if precision_mass == 0:
    pass
else:
    M_pl, M_pl_err = df['pl_bmasse'].values, (abs(df['pl_bmasseerr1'].values) + abs(df['pl_bmasseerr1'].values)) / 2
    idxs_M_pl_thres = np.where(M_pl_err / M_pl < (precision_mass / 100))[0]
    df = df.iloc[idxs_M_pl_thres]
    
if precision_radius == 0:
    pass
else:
    R_pl, R_pl_err = df['pl_rade'].values, (abs(df['pl_radeerr1'].values) + abs(df['pl_radeerr2'].values)) / 2
    idxs_R_pl_thres = np.where(R_pl_err / R_pl < (precision_radius / 100))[0]
    df = df.iloc[idxs_R_pl_thres]

In [ ]:
if catalog == 'PlanetS' or catalog == 'Exoplanet.eu':
    fac_R = const.R_jup.value / const.R_earth.value
else:
    fac_R = 1

P_catalog = df['pl_orbper'].values
R_catalog = df['pl_rade'].values * fac_R

In [ ]:
#@|-----------------------------------------------------------------
#@|Boundaries of the Neptunian desert (Castro-González et al. 2024)
#@|-----------------------------------------------------------------

x_tw_low = np.linspace(-0.30, np.log10(2.97), 100)
x_tw_low = np.linspace(-0.30, 0.47, 100)
x_tw_up = np.linspace(0.120, np.log10(2.97), 100)
x_tw_up = np.linspace(0.120, 0.473, 100)

y_tw_up = -0.43 * x_tw_up + 1.14
y_tw_low = +0.55 * x_tw_low + 0.36

In [ ]:
#@|------------------
#@|-------PLOT-------
#@|------------------

fig, ax = plt.subplots(figsize=(8, 7))
text_list = [str(x.strip()) for x in text.split(',')]
boundaries_list = [str(x.strip()) for x in boundaries.split(',')]

#@|data
ax.scatter(x=np.log10(P_catalog), y=np.log10(R_catalog), \
            ec = 'white', s = 30, zorder = 1, fc = 'grey', alpha = 1, lw = 0.7)


if 'desert' in boundaries_list:

    ax.vlines(x=[np.log10(2.95), -0.30, 0.12], ymin=[np.log10(4.1), -0.07, 1.09],\
               ymax=[np.log10(8.4), 0.20, 1.20], colors = color_boundaries, \
               linestyle = style_boundaries, lw = 2.3, zorder = 1e10000)

    ax.plot(x_tw_up, y_tw_up,  c = color_boundaries, linestyle = style_boundaries, lw = 2.3, zorder = 1e1000)
    ax.plot(x_tw_low, y_tw_low,  c = color_boundaries, linestyle = style_boundaries, lw = 2.3, zorder = 1e1000)

if 'ridge' in boundaries_list:

    ax.vlines(x=[np.log10(2.95), np.log10(5.7)], ymin=[np.log10(4.1), np.log10(5)],\
           ymax=[np.log10(8.4),np.log10(7.4)], colors = color_boundaries, \
           linestyle = style_boundaries, lw = 2.3, zorder = 1e10000)
    

if 'ridge' in boundaries_list and 'desert' in boundaries_list:

    ax.vlines(x=[np.log10(2.95), -0.30, 0.12, np.log10(5.7)], ymin=[np.log10(4.1), -0.07, 1.09, np.log10(5)],\
           ymax=[np.log10(8.4), 0.20, 1.20, np.log10(7.4)], colors = color_boundaries, \
           linestyle = style_boundaries, lw = 2.3, zorder = 1e10000)

    ax.plot(x_tw_up, y_tw_up,  c = color_boundaries, linestyle = style_boundaries, lw = 2.3, zorder = 1e1000)
    ax.plot(x_tw_low, y_tw_low,  c = color_boundaries, linestyle = style_boundaries, lw = 2.3, zorder = 1e1000)


# 2D density plot of the observed planet population

if catalog == 'NEA':

    sns.kdeplot(x=np.log10(P_catalog), y=np.log10(R_catalog), cmap = color_map, \
                fill = True, zorder = -1, alpha = 1, bw_adjust= 0.3,  linewidths = 2.2, \
                levels = [0,  0.163, 0.165, 0.17, 0.18, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, ])

if catalog == 'Exoplanet.eu':
    sns.kdeplot(x=np.log10(P_catalog), y=np.log10(R_catalog), cmap = color_map, \
                fill = True, zorder = -1, alpha = 1, bw_adjust= 0.3,  linewidths = 2.2, \
                levels = [0, 0.19, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6,  ])
    
if catalog == 'PlanetS':
    sns.kdeplot(x=np.log10(P_catalog), y=np.log10(R_catalog), cmap = color_map, \
                fill = True, zorder = -1, alpha = 1, bw_adjust= 0.3,  linewidths = 2.2, \
                levels = [0, 0.17, 0.18, 0.19, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
    

if 'desert' in text_list:
    txt=plt.text(-0.25, 0.74,'DESERT',**{'fontname':'Arial Black'},zorder=100,fontsize=20,alpha=0.8,color=color_text, fontweight = 600)
    txt.set_path_effects([PathEffects.withStroke(linewidth=2, foreground='black')])
    
if 'ridge' in text_list:
    txt=plt.text(0.58, 0.65, 'RIDGE',rotation=90,**{'fontname':'Arial Black'},zorder=100,fontsize=20,alpha=0.8,color=color_text,  fontweight = 600)
    txt.set_path_effects([PathEffects.withStroke(linewidth=2, foreground='black')])

if 'savanna' in text_list:
    
    txt=plt.text(1.12, 0.74,'SAVANNA',**{'fontname':'Arial Black'},zorder=100,fontsize=20,alpha=0.8,color=color_text, fontweight = 600)
    txt.set_path_effects([PathEffects.withStroke(linewidth=2, foreground='black')]) 



for i in range(1, 21):
    
    try:
        
        if legend_or_box == 'legend':

            ax.scatter(np.log10(globals()['p_p'+str(i)]), np.log10(globals()['r_p'+str(i)]),\
                        fc = globals()['color_p'+str(i)], ec = 'k', s = 150, zorder = 1, marker = 'h',\
                       label = globals()['name_p'+str(i)])
            
            up = np.log10(globals()['r_p'+str(i)] + globals()['r_p'+str(i)+'_err_up'])-\
                                                                np.log10(globals()['r_p'+str(i)])
            down =  np.log10(globals()['r_p'+str(i)]) - np.log10(globals()['r_p'+str(i)]-\
                                                                 globals()['r_p'+str(i)+'_err_down']) 

            ax.errorbar(np.log10(globals()['p_p'+str(i)]), np.log10(globals()['r_p'+str(i)]), \
                        yerr = (np.array([down]),np.array([up])), \
                         c = 'k', lw = 2, zorder = 0)
            
        else:
            
            ax.scatter(np.log10(globals()['p_p'+str(i)]), np.log10(globals()['r_p'+str(i)]),\
                        fc = globals()['color_p'+str(i)], ec = 'k', s = 150, zorder = 1, marker = 'h')
            
            up = np.log10(globals()['r_p'+str(i)] + globals()['r_p'+str(i)+'_err_up'])-\
                                                                np.log10(globals()['r_p'+str(i)])
            down =  np.log10(globals()['r_p'+str(i)]) - np.log10(globals()['r_p'+str(i)]-\
                                                                 globals()['r_p'+str(i)+'_err_down']) 

            ax.errorbar(np.log10(globals()['p_p'+str(i)]), np.log10(globals()['r_p'+str(i)]), \
                        yerr = (np.array([down]),np.array([up])), \
                         c = 'k', lw = 2, zorder = 0)
            
            ax.text(np.log10(globals()['p_p'+str(i)]+ globals()['dis_x_p'+str(i)]),\
                     np.log10(globals()['r_p'+str(i)]+  globals()['dis_y_p'+str(i)]),\
                     globals()['name_p'+str(i)], fontsize=10.5, \
                    verticalalignment = 'bottom', horizontalalignment = 'center', \
                     bbox = {"boxstyle": "round", 'facecolor': globals()['color_p'+str(i)], \
                             'alpha': 0.7, 'pad': 0.4}, \
                     c = 'white', weight='bold', zorder = 10000)
        
    except:
        pass
    
    

ax.set_xticks([-0.4,  0, 0.4, 0.8,1.2, 1.6,\
               2.0, 2.4 ],\
              [round(10**-0.4, 1), '1.0',\
              round(10**0.4, 1), round(10**0.8, 1),\
              round(10**1.2, 1), round(10**1.6, 1),\
              round(10**2.0, 1), round(10**2.4, 1)], fontsize = 13.5)



ax.set_yticks([0,0.2,0.4,0.6,0.8, 1.0, 1.2, 1.4], \
              ['1.0',round(10**0.2, 1),\
              round(10**0.4, 1),round(10**0.6, 1),\
              round(10**0.8, 1), round(10**1.0, 1),\
              round(10**1.2, 1), round(10**1.4, 1)], fontsize = 13.5)    


ax.set_xlim(-0.5, 2.2)
ax.set_ylim(0.07, 1.36)

ax.set_xlabel('Orbital period (days)', fontsize = 16)
ax.set_ylabel(r'Planet radius ($\rm R_{\oplus}$)', fontsize = 16)

ax.tick_params(which='major', length = 6, color = 'k', direction = 'in', width = 1.3)

if legend_or_box == 'legend':
    ax.legend(fontsize = 12, loc = 'upper right', markerscale = 0.8)
else:
    pass

#@|+++++++++++
#@|Save Figure
#@|+++++++++++

plt.savefig(f'output/nep_des_{plot_name}.pdf', bbox_inches = 'tight')
plt.savefig(f'output/nep_des_{plot_name}.png', bbox_inches = 'tight', dpi = 400)
#plt.savefig(f'output/nep_des_{plot_name}.jpg', bbox_inches = 'tight', dpi = 400)
